# Re-ID Backbone Comparison

This notebook demonstrates how to compare multiple appearance backbones (e.g. `osnet`, `dinov2_vitl14`) using the `tools/eval_reid.py` utility.

In [ ]:
import json
import pathlib
import subprocess
import pandas as pd

PROJECT_ROOT = pathlib.Path('..').resolve()
EVAL_SCRIPT = PROJECT_ROOT / 'tools' / 'eval_reid.py'
RESULTS_DIR = PROJECT_ROOT / 'artifacts' / 'reid_embeddings'
BACKBONES = ['osnet', 'dinov2_vitl14', 'clip_vitl14']


## Evaluate mAP/CMC

Embeddings are assumed to be stored in ``RESULTS_DIR / f"{backbone}_query.npz"`` and ``RESULTS_DIR / f"{backbone}_gallery.npz"``.

In [ ]:
records = []
for backbone in BACKBONES:
    query_file = RESULTS_DIR / f'{backbone}_query.npz'
    gallery_file = RESULTS_DIR / f'{backbone}_gallery.npz'
    if not query_file.exists() or not gallery_file.exists():
        print(f'Skipping {backbone}: missing embedding files')
        continue
    completed = subprocess.run(
        ['python', str(EVAL_SCRIPT), 'reid', '--query', str(query_file), '--gallery', str(gallery_file)],
        capture_output=True,
        check=True,
        text=True,
    )
    metrics = json.loads(completed.stdout)
    record = {'backbone': backbone, 'mAP': metrics['mAP']}
    # Populate a few standard ranks
    for rank_key, value in metrics['CMC'].items():
        if rank_key in {'rank-1', 'rank-5', 'rank-10'}:
            record[rank_key] = value
    records.append(record)

comparison = pd.DataFrame.from_records(records)
comparison.sort_values('mAP', ascending=False, inplace=True)
comparison.reset_index(drop=True, inplace=True)
comparison

## Visualize retrieval examples

Once the backbone is chosen, we can inspect top retrievals.

In [ ]:
import numpy as np

def rank_gallery(query_feat: np.ndarray, gallery_feats: np.ndarray) -> np.ndarray:
    query = query_feat / np.linalg.norm(query_feat)
    gallery = gallery_feats / np.linalg.norm(gallery_feats, axis=1, keepdims=True)
    return np.argsort(gallery @ query)[::-1]

# Example usage (paths to be adapted)
# query_npz = RESULTS_DIR / 'osnet_query.npz'
# gallery_npz = RESULTS_DIR / 'osnet_gallery.npz'
# query_data = np.load(query_npz)
# gallery_data = np.load(gallery_npz)
# order = rank_gallery(query_data['features'][0], gallery_data['features'])
# order[:10]